In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the Data

df_custom = pd.read_csv('Custom_Crops_yield_Historical_Dataset.csv')
df_prod   = pd.read_csv('crop_production.csv')

print(f"Custom shape: {df_custom.shape}")
print(f"Production shape: {df_prod.shape}")

Custom shape: (50765, 20)
Production shape: (246091, 7)


In [3]:
# Rename columns in production data for merging

df_prod = df_prod.rename(columns={
    'State_Name': 'State Name',
    'District_Name': 'Dist Name',
    'Crop_Year': 'Year',
    'Crop': 'Crop'
})

In [4]:
# Clean strings for reliable merging

def clean_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip().lower().replace("  ", " ").replace("&", "and").replace("  ", " ")

for col in ['State Name', 'Dist Name', 'Crop']:
    df_custom[col] = df_custom[col].apply(clean_str)
    df_prod[col]   = df_prod[col].apply(clean_str)

In [5]:
# Merge (Custom as base + season data from production)

merged = pd.merge(
    df_custom,
    df_prod[['State Name', 'Dist Name', 'Year', 'Crop', 'Season']],
    on=['State Name', 'Dist Name', 'Year', 'Crop'],
    how='left'          # keep all rows from Custom
)

print(f"Merged shape: {merged.shape}")
print("Seasons added:", merged['Season'].notna().sum())

Merged shape: (55067, 21)
Seasons added: 11206


In [6]:
# Basic cleaning & Validation

# Drop rows with invalid yield or area
merged = merged[merged['Yield_kg_per_ha'] > 0]
merged = merged[merged['Area_ha'] > 0]

# Fill missing Season with "Unknown"
merged['Season'] = merged['Season'].fillna('Unknown')

# Convert Year to int
merged['Year'] = merged['Year'].astype(int)

In [7]:
# Remove redundant/derived columns

# Total_N_kg etc. are just Area * req → we keep the req features (more interpretable)
cols_to_drop = ['Total_N_kg', 'Total_P_kg', 'Total_K_kg']
merged = merged.drop(columns=cols_to_drop, errors='ignore')

In [8]:
# Outlier Handling (reasonable bounds for Indian crops)

# Yield outliers (most crops 200–8000 kg/ha)
merged = merged[(merged['Yield_kg_per_ha'] >= 50) & (merged['Yield_kg_per_ha'] <= 12000)]

# Area outliers (remove tiny unrealistic records)
merged = merged[merged['Area_ha'] >= 1]

print(f"After cleaning: {merged.shape}")

After cleaning: (54804, 18)


In [9]:
# Feature Engineering

# Group rare crops into "Other" (for better model stability)
crop_counts = merged['Crop'].value_counts()
rare_crops = crop_counts[crop_counts < 50].index
merged['Crop'] = merged['Crop'].replace(rare_crops, 'Other')

# Create a simple "Decade" feature (optional but useful for trends)
merged['Decade'] = (merged['Year'] // 10) * 10

In [10]:
# Encode Categorical Features

# For XGBoost we can use LabelEncoder or leave as string (XGBoost handles strings in newer versions)
# Here we create encoded versions for safety
le_crop = LabelEncoder()
le_state = LabelEncoder()
le_season = LabelEncoder()

merged['Crop_encoded'] = le_crop.fit_transform(merged['Crop'])
merged['State_encoded'] = le_state.fit_transform(merged['State Name'])
merged['Season_encoded'] = le_season.fit_transform(merged['Season'])

In [11]:
# Final column selection and saving

final_cols = [
    'Year', 'Decade', 'State Name', 'State_encoded', 'Dist Name',
    'Crop', 'Crop_encoded', 'Season', 'Season_encoded',
    'Area_ha',
    'N_req_kg_per_ha', 'P_req_kg_per_ha', 'K_req_kg_per_ha',
    'Temperature_C', 'Humidity_%', 'pH', 'Rainfall_mm',
    'Wind_Speed_m_s', 'Solar_Radiation_MJ_m2_day',
    'Yield_kg_per_ha'          # <-- TARGET
]

final_df = merged[final_cols].copy()

# Save cleaned merged dataset
final_df.to_csv('merged_preprocessed_crop_yield.csv', index=False)

print("Final preprocessed dataset saved as 'merged_preprocessed_crop_yield.csv'")
print(final_df.head())
print(final_df.info())

Final preprocessed dataset saved as 'merged_preprocessed_crop_yield.csv'
   Year  Decade    State Name  State_encoded Dist Name      Crop  \
0  1966    1960  chhattisgarh              3      durg      rice   
1  1966    1960  chhattisgarh              3      durg     maize   
2  1966    1960  chhattisgarh              3      durg  chickpea   
3  1967    1960  chhattisgarh              3      durg      rice   
4  1967    1960  chhattisgarh              3      durg     maize   

   Crop_encoded   Season  Season_encoded   Area_ha  N_req_kg_per_ha  \
0             3  Unknown               4  548000.0          8.43975   
1             2  Unknown               4    3000.0         18.00009   
2             0  Unknown               4   54000.0          9.00000   
3             3  Unknown               4  547000.0         18.69275   
4             2  Unknown               4    3000.0         27.00000   

   P_req_kg_per_ha  K_req_kg_per_ha  Temperature_C  Humidity_%   pH  \
0          4.05108  